<a href="https://colab.research.google.com/github/w3c/sustyweb/blob/main/Web_Sustainability_GRI_Impact_est.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook tries to assess environmental impact of W3C Sustainable Web recommendations across a number of categories aligned with GRI reporting.

As an input it takes a spreadsheet with recommendations and their indicators of impact on reduction of server resource usage, network transfer and end user device usage.
Then it takes data from greenIT.fr report which estimates environmental impact of above categories across material use, water use, energy use and GHG emissions.
It then combines these datasets and estimates comparative impact of given recomendation on different sections of GRI taxonomy.

In [1]:
import gspread
import pandas as pd
from google.colab import auth
from google.colab import files
from google.auth import credentials
from google.auth import default



In [2]:
# All the recommendations in the spreadsheet are scored in terms of their impact on end user devices, network and servers in data centre
# Scores are from 0 to MAX_THRESHOLD, by default 2
# If maximum score in the spreadsheet is changed, it needs to updated here
MAX_THRESHOLD = 2

# Labels to be assigned to metrics
labels = {'Low': MAX_THRESHOLD / 3 ,
          'Moderate': MAX_THRESHOLD / 3 * 2,
          'High': MAX_THRESHOLD }

# Set to missing if assessment undefined
MISSING = 'Missing'

In [3]:
# Auth and read the spreadsheet into pandas
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_url = 'https://docs.google.com/spreadsheets/d/12nGydnSv24fvmvCM-665_pFGPG9u3RgTwe1sCz4eiGk'
df = pd.read_csv(sheet_url + '/export?gid=0&format=csv', skiprows=[0, 1])

df.head(5)

,Guideline,Spec,Tags,Data Centre,Network,Device,Comments
0,Undertake Systemic Impacts Mapping,UX,Social Equity\n Accessibility\n Ideation\n Res...,1,1,1,NaN
1,Assess And Research Visitor Needs,UX,Social Equity\n Accessibility\n UI\n Patterns\...,1,1,1,NaN
2,Research Non-visitors Needs,UX,Social Equity\n Ideation\n Research\n Usabilit...,1,1,1,NaN
3,Consider Sustainability In Early Ideation,UX,UI\n Ideation\n Research\n Software\n Strategy,0,0,0,NaN
4,Account For Stakeholder Issues,UX,Social Equity\n Accessibility\n Ideation\n Res...,1,1,1,NaN


In [4]:

# Impact factors across number of areas split by end user devices, network equipment and data centers
# Source: GreenIT.fr: https://www.greenit.fr/wp-content/uploads/2019/11/GREENIT_EENM_etude_EN_accessible.pdf
data = [  [.6, .63, .83, .75]
        , [.23, .22, .09, .16]
        , [.17, .15, .07, .08] ]

factors = pd.DataFrame(data, columns=['Energy', 'Emissions', 'Water', 'Materials'],
                       index=['Device', 'Network', 'Data Centre'])

factors.head()


,Energy,Emissions,Water,Materials
Device,0.60,0.63,0.83,0.75
Network,0.23,0.22,0.09,0.16
Data Centre,0.17,0.15,0.07,0.08


In [5]:
# Calculate impacts (numerical)

# GRI-301: Materials
df['GRI-301'] = df['Data Centre'] * factors['Materials']['Data Centre'] \
              + df['Network'] * factors['Materials']['Network'] \
              + df['Device'] * factors['Materials']['Device']

# GRI-302: Enegry
df['GRI-302'] = df['Data Centre'] * factors['Energy']['Data Centre'] \
              + df['Network'] * factors['Energy']['Network'] \
              + df['Device'] * factors['Energy']['Device']

# GRI-303: Water
df['GRI-303'] = df['Data Centre'] * factors['Water']['Data Centre'] \
              + df['Network'] * factors['Water']['Network'] \
              + df['Device'] * factors['Water']['Device']

# GRI-305: Emissions
df['GRI-305'] = df['Data Centre'] * factors['Emissions']['Data Centre'] \
              + df['Network'] * factors['Emissions']['Network'] \
              + df['Device'] * factors['Emissions']['Device']



In [6]:
# Assign labels to calculated impacts
# If assessment for a given recommendation is missing it will be set to MISSING
df['GRI-301 Label'] = df.apply(lambda x: 'Low' if x['GRI-301'] < labels['Low']
                                               else ('Moderate' if x['GRI-301'] < labels['Moderate']
                                                                else 'High' if x['GRI-301'] <= labels['High']
                                                                            else MISSING), axis=1 )

df['GRI-302 Label'] = df.apply(lambda x: 'Low' if x['GRI-302'] < labels['Low']
                                               else ('Moderate' if x['GRI-302'] < labels['Moderate']
                                                                else 'High' if x['GRI-302'] <= labels['High']
                                                                            else MISSING), axis=1 )

df['GRI-303 Label'] = df.apply(lambda x: 'Low' if x['GRI-303'] < labels['Low']
                                               else ('Moderate' if x['GRI-303'] < labels['Moderate']
                                                                else 'High' if x['GRI-303'] <= labels['High']
                                                                            else MISSING), axis=1 )

df['GRI-305 Label'] = df.apply(lambda x: 'Low' if x['GRI-305'] < labels['Low']
                                               else ('Moderate' if x['GRI-305'] < labels['Moderate']
                                                                else 'High' if x['GRI-305'] <= labels['High']
                                                                            else MISSING), axis=1 )


df.head()

,Guideline,Spec,Tags,Data Centre,Network,Device,Comments,GRI-301,GRI-302,GRI-303,GRI-305,GRI-301 Label,GRI-302 Label,GRI-303 Label,GRI-305 Label
0,Undertake Systemic Impacts Mapping,UX,Social Equity\n Accessibility\n Ideation\n Res...,1,1,1,NaN,0.99,1.0,0.99,1.0,Moderate,Moderate,Moderate,Moderate
1,Assess And Research Visitor Needs,UX,Social Equity\n Accessibility\n UI\n Patterns\...,1,1,1,NaN,0.99,1.0,0.99,1.0,Moderate,Moderate,Moderate,Moderate
2,Research Non-visitors Needs,UX,Social Equity\n Ideation\n Research\n Usabilit...,1,1,1,NaN,0.99,1.0,0.99,1.0,Moderate,Moderate,Moderate,Moderate
3,Consider Sustainability In Early Ideation,UX,UI\n Ideation\n Research\n Software\n Strategy,0,0,0,NaN,0.00,0.0,0.00,0.0,Low,Low,Low,Low
4,Account For Stakeholder Issues,UX,Social Equity\n Accessibility\n Ideation\n Res...,1,1,1,NaN,0.99,1.0,0.99,1.0,Moderate,Moderate,Moderate,Moderate


In [7]:
# Save to excel and download locally

df.to_excel('Recommendations scored.xlsx')

files.download('Recommendations scored.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>